In [14]:
import re
from pathlib import Path
import numpy as np 
import pandas as pd
import matplotlib.pyplot
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm
import os

In [22]:
input_path = Path.cwd()/('./Breast Cancer') 
path_contents = [path for path in Path.iterdir(input_path)]
print(path_contents)

[WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10253'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10254'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10255'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10256'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10257'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10258'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10259'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10260'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10261'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10262'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10264'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10268'), WindowsPath('D:/projects/breast cancer classification/Breast Cancer/10269')

In [23]:
def get_files_folders(input_path,print_summary = True):
    folders = []
    files = []
    for item in Path.iterdir(path):
        if item.is_dir():
            folders.append(item)
        else:
            files.append(item)
    if print_summary:
        print(f'There are {len(folders)} folders and {len(files)} files in folder {path.name}') 
    
    return {'folders': folders, 'files': files, 'path': path} 

In [24]:
sorted_paths = get_files_folders(input_path)

patient_1 = sorted_paths['folders'][0]

for p in Path.iterdir(patient_1):
    print(p,'\n')
# In order to obtain the number of files and folders
get_files_folders(patient_1);


There are 280 folders and 2 files in folder Breast Cancer
D:\projects\breast cancer classification\Breast Cancer\10253\0 

D:\projects\breast cancer classification\Breast Cancer\10253\1 

There are 280 folders and 2 files in folder Breast Cancer


In [27]:
def verify_path_contents(input_path,n_folders,n_files,print_summary = True):
    '''
    loops through all the directories in a path and checks the number of folders and files in each directory
    returns a dictionary containing a list of the matched paths and a list of the unmatched paths
    '''
    
    matched_paths = []
    unmatched_paths = []
    
    for path in Path.iterdir(input_path):
        if path.is_dir():
            folders = [sub_path for sub_path in Path.iterdir(path) if sub_path.is_dir()]
            files = [sub_path for sub_path in Path.iterdir(path) if sub_path.is_file()]

            if len(folders) != n_folders or len(files) != n_files:
                unmatched_paths.append(path)
            else:
                matched_paths.append(path)
        
    if print_summary:
        print(f'We have a total of {len(matched_paths)} matched paths, and a total of {len(unmatched_paths)} unmatched paths in {input_path.name}')
        
    return {'matched_paths':matched_paths,'unmatched_paths':unmatched_paths}

In [28]:
path_contents = verify_path_contents(input_path = input_path, n_folders = 2, n_files = 0)


We have a total of 279 matched paths, and a total of 1 unmatched paths in Breast Cancer


In [45]:
patient_1_paths = get_files_folders(patient_1)


There are 280 folders and 2 files in folder Breast Cancer


In [46]:
p = {}
# store the paths of each folder, by using it's name, in the dictionary above 
for folder in patient_1_paths['folders']:
    p[folder.name] = get_files_folders(folder)

There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 

There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 folders and 2 files in folder Breast Cancer
There are 280 

In [47]:
# displaying the first 5 paths in each directory
display(p['0']['files'][0:5])
display(p['1']['files'][0:5])

KeyError: '0'

###  Storing the image_path, patient_id, target and x & y coordinates
#### We have about 278,000 images. To feed the algorithm with image patches we will store the path of each image and create a dataframe containing all the paths. This way we can load batches of images one by one without storing the individual pixel values of all images.

#### Now that we know our basic file structure which seems to be as follows:

breast-histopathology-images:
-->patient_id:
-->-->cancer_class:
-->-->-->image_files

### we need this infos :
patient_id , 
x_coordinate , 
y_coordinate , 
target , 
path_to_image 

In [36]:
def extract_all_image_paths(input_path = Path.cwd()/'../input/breast-histopathology-images'):
    image_paths = [image_path for image_path in Path.glob(input_path,pattern = '*/*/*.png')]
    return image_paths

In [54]:
image_paths = extract_all_image_paths()

In [55]:
image_paths[0:5]

[]

In [56]:
print(f'We have a total of {len(image_paths)} images.')


We have a total of 0 images.


In [57]:
# let's check that we correctly extracted our paths
display(image_paths[0:3])
display(image_paths[10000:10003])

[]

[]

### Extracting Metadata into DataFrame¶


In [58]:
# Extracts a dict of lists containing the informaion of each path
def extract_metadata(image_paths) -> dict:
    path_data = {'path':[],'patient_id':[],'x_coord':[] ,'y_coord':[],'target':[]}
    pattern = '\/(\d+)_.+_x(\d+)_y(\d+)_.+(\d)'
    for image_path in tqdm(image_paths,total = 277524):
        meta_data = re.search(pattern, str(image_path))
        path_data['path'].append(image_path)
        path_data['patient_id'].append(meta_data.group(1))
        path_data['x_coord'].append(meta_data.group(2))
        path_data['y_coord'].append(meta_data.group(3))
        path_data['target'].append(meta_data.group(4))
    return path_data

In [59]:
path_data = extract_metadata(image_paths)
# extract and display the first 3 rows of each value
for value in path_data.values():
    display(value[0:3]) 

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [62]:
# convert dictionary to pandas dataframe --> convert path_data dict to dataframe
df = pd.DataFrame.from_dict(path_data
                            
df.head()
                            
df.info()

SyntaxError: invalid syntax (<ipython-input-62-a7a5cbd94a1d>, line 4)

In [63]:
df.iloc[:,2:5]
for col in df.iloc[:,2:5]:
    df[col] = df[col].astype('int')

Exception ignored in: <function tqdm.__del__ at 0x00000209C3A9E8C8>
Traceback (most recent call last):
  File "c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py", line 1131, in __del__
    self.close()
  File "c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\tqdm\notebook.py", line 261, in close
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignored in: <function tqdm.__del__ at 0x00000209C3A9E8C8>
Traceback (most recent call last):
  File "c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py", line 1131, in __del__
    self.close()
  File "c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\tqdm\notebook.py", line 261, in close
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


NameError: name 'df' is not defined

## Exploratory Data Analysis (EDA)

## Analytics:
How many patches do we have on average per patient?

On average how many of those patches are cancerous vs non-cancerous?

Let's try and answer those questions by querying our dataframe.

In [64]:
# Function below gets the the number of patches for each patient, 
# and gets the ratio of cancerous patches to non-cancerous patches for each patient

def get_image_count_and_cancer_ratios(path_df):
    # Return the count of images per patient
    s1 = path_df.groupby('patient_id')['target'].count()
    # Return the ratio of cancerous to non-cancerous images per-patient
    s2 = path_df.groupby('patient_id')['target'].mean()

    # Merge the series into one dataframe that uses the same index
    df_summary = pd.concat([s1,s2],axis = 1)
    df_summary.columns = ['n_patches/patient','cancer_ratio']
    return df_summary

In [65]:
df_summary = get_image_count_and_cancer_ratios(path_df = df)
display(df_summary)

NameError: name 'df' is not defined

In [66]:
# set bin values for number of images
bin_values = np.linspace(0,2400,25)
# below we will group our data into bins in order to e
def get_binned_cancer_ratio_df(path_df,bin_values):
    df_summary = get_image_count_and_cancer_ratios(path_df)
    # use cut to determine which values fit into which bin
    bins = pd.cut(df_summary['n_patches/patient'], bin_values)
    # group your dataframe by the bins
    binned_df = df_summary.groupby(bins).median() 
    binned_df['non-cancer_ratio_median'] = binned_df['cancer_ratio'].apply(lambda x: 1-x)
    binned_df['n_patients_per_bin'] = df_summary['n_patches/patient'].groupby(bins).count()
    # rename cancer ratio to cancer ratio median since we are taking the median now
    binned_df.rename(columns={'cancer_ratio':'cancer_ratio_median'},inplace=True)
    return binned_df

In [67]:
binned_df = get_binned_cancer_ratio_df(df,bin_values)
binned_df # fix the average number n_imgs/patient

NameError: name 'df' is not defined

In [68]:
def plot_patch_info(binned_df,bin_values):
    
    # get the the ranges for the x-axis since this is supposed to emulate a histogram
    ranges = []
    for i in range(len(bin_values)):
        try: r = '('+str(int(bin_values[i]))+'-'+str(int(bin_values[i+1]))+')'
        except: pass
        ranges.append(r)
    
    # Number of Patients per Bin -> Bar Plot
    trace_0 = go.Bar(name='Number of Patients Per Bin',x=ranges, y=binned_df['n_patients_per_bin'],marker_color='#ff6efa');
    # Ratio of Cancerous Images -> Bar Plot
    trace_1 = go.Bar(name='Cancerous Images', x=ranges, y=binned_df['cancer_ratio_median'],marker_color='#8634eb');
    # Ratio of Non-Cancerous Images -> Bar Plot
    trace_2 = go.Bar(name='Non-Cancerous Images', x=ranges, y=binned_df['non-cancer_ratio_median'],marker_color='#ff87c9')
    # Number of Patches per Patient vs Cancer Ratio of Image -> Scatter Plot
    trace_3 = go.Scatter(name='# Patches vs Cancer Ratio ',x = df_summary['n_patches/patient'],y = df_summary['cancer_ratio'],mode='markers',marker=dict(size=16,color=df_summary['cancer_ratio'], colorscale=[[0.0, "#ff87c9"],[1.0, "#8634eb"]]))
    fig = make_subplots(rows=3, cols=1, 
                        shared_xaxes=True, 
                        vertical_spacing=0.05,
                        subplot_titles=("Frequency of Number of Patches",
                                        "Ratio of Cancerous to Non Cancerous Images as a Function of Number of Images per Patient",
                                        "Number of Patches per Patient vs Cancer Ratio of Image")
                       )

    # Change the bar mode
    fig.update_layout(
        barmode='stack',
        bargap=0
    )

    fig.update_yaxes(title_text="Frequency", row=1, col=1)
    fig.update_yaxes(title_text="Percentage", row=2, col=1)
    fig.update_yaxes(title_text="Percentage", row=3, col=1)

    fig.update_xaxes(title_text="Number of Patches", row=3, col=1)
        fig.update_layout(height=1800,legend={'traceorder':'normal'})
    fig.update_xaxes(rangemode="tozero")

    fig.append_trace(trace_0,1,1)
    fig.append_trace(trace_1,2,1)
    fig.append_trace(trace_2,2,1)
    fig.append_trace(trace_3,3,1)


    fig.show()

IndentationError: unexpected indent (<ipython-input-68-2316dc94b1b5>, line 37)

In [69]:
IPython.OutputArea.auto_scroll_threshold = 130;

plot_patch_info(binned_df,bin_values) # put image in next cell hidden in case image does not show 


NameError: name 'IPython' is not defined

In [70]:
plt.figure(figsize = (50,50));
io.imshow('../input/plot-images/patch_info.jpg');

NameError: name 'plt' is not defined

In [71]:
# read image from path
img = io.imread('../input/breast-histopathology-images/10253/0/10253_idx5_x1001_y1001_class0.png')
# show the image after being read
io.imshow(img);

NameError: name 'io' is not defined

In [72]:
print(img.shape)


NameError: name 'img' is not defined

In [73]:
# Accessing the first row
first_row = img[0:1]
print('FIRST ROW:')
print(f'The shape of the first row: {first_row.shape}')
print(f'The first five pixels of the first row: \n\n{first_row[0:1,0:5]}\n')
# Accessing the first column
first_column = img[:,0:1]
print('FIRST COLUMN:')
print(f'The shape of the first column: {first_column.shape}')
print(f'The first five pixels of the first row: \n\n{first_column[0:5,0:1]}\n')
# Accessing a specific pixel
pixel = img[19:20,16:17]
print('RGB VALUE 20th ROW, 17th COLUMN:')
print(f'The shape of the pixel: {pixel.shape}')
print(f'The first five pixels of the first row: \n\n{pixel}\n')

NameError: name 'img' is not defined

In [74]:
fig,axs = plt.subplots(1,3,figsize = (30,10));
# Displaying the first row
axs[0].imshow(first_row);
# Displaying the first column
axs[1].imshow(first_column);
#Displaying the pixel
axs[2].imshow(pixel);

NameError: name 'plt' is not defined

In [75]:
display(img[0:1].shape)
display(img[0:1].ndim)
display(img[0].shape)
display(img[0].ndim)

NameError: name 'img' is not defined

In [76]:
fig,axs = plt.subplots(1,3,figsize = (20,50));
axs[0].imshow(img[:,0]);
axs[1].imshow(img[:,0:1]);
# using reshape to achieve the same result
axs[2].imshow(img[:,0].reshape(50,1,3));

NameError: name 'plt' is not defined

## Healthy Tissue Patches Vs Cancerous Tissue Patches¶


In [78]:
path_df = df



def show_patches(target='Any',nrows = 2,ncols = 5, path_df = path_df):
    '''
    Function accepts type of target:
    target = 0 --> Non-Cancerous Tissue
    target = 1 --> Cancerous Tissue
    path_df: dataframe containing paths
    '''
    if target == 'Any':
        # if target is set to any retrieve both cancerous and non-cancerous cells
        tissue_indices = np.random.choice(path_df.index, size=nrows*ncols, replace=False)
    else:
        # replace = False means that no duplication is allowed
        tissue_indices = np.random.choice(path_df[path_df['target']==target].index, size=nrows*ncols, replace=False)
    
    fig,axes = plt.subplots(nrows=nrows,ncols=ncols,figsize = (ncols*6,nrows*6))
            
    for i,ax in enumerate(axes.flatten()):
        idx = tissue_indices[i]
        img = io.imread(path_df.loc[idx,'path'])
        ax.imshow(img)

NameError: name 'df' is not defined

## non-cancerous

show_patches(target=0,ncols=10)


## cancerous

In [80]:
show_patches(target=1,ncols=10)

NameError: name 'show_patches' is not defined

## Visualizing the Breast Tissue


In [81]:
def get_patient_df(p_id,df=df):
    return df.loc[df['patient_id']== p_id,:] 

NameError: name 'df' is not defined

In [82]:
from matplotlib.colors import LinearSegmentedColormap

def scatter_patient_xy(nrows=3,ncols=3):
    n_imgs = nrows*ncols
    # get random patient ids to plot
    p_ids = np.random.choice(df['patient_id'].unique(), size=n_imgs, replace=False)
    
    fig, axs = plt.subplots(nrows=nrows,ncols=ncols,figsize=(30,30))
    
    colors = ['#ff70db','#9334eb']
    
    # setting the point colors for the target values
    cmap = LinearSegmentedColormap.from_list(name='',colors = colors, N=2)
    
    for i,row in enumerate(axs):
        for j,ax in enumerate(row):
            p_id = p_ids[i*nrows+j]
            p_df = get_patient_df(p_id)
            
            x_coords = p_df['x_coord'].values
            y_coords = p_df['y_coord'].values
            
            # get the min coordinate of each patient image
            min_coord = min(x_coords.max(),y_coords.max())
            # use min_coord to to set the size of the plotted square
            s = int(4000/min_coord*15)
            
            # used to determine the color of the point
            targets = p_df['target'].values
            ax.scatter(x_coords,y_coords,c=targets,cmap=cmap,s=s,marker='s')
            ax.set_title('Patient: '+p_id)

In [83]:
scatter_patient_xy()


NameError: name 'df' is not defined

In [84]:
def get_tissue_image_array(patient_id,pred = False):
    # get patient dataframe
    path_df = get_patient_df(patient_id)
    # get the max_coordinate to define the numpy array size, numpy array will be containing the rgb values for the image
    max_coord = np.max((*path_df['x_coord'],*path_df['y_coord']))
    # add 50 to the max_coord to get the final image dimension,50 here is the patch size
    image_dimension = max_coord + 50
    # create a 3 dimensional array with RGB values = 255, we set the type as uint since no RGB value can be > 255, thus increasing efficiency
    grid = 255*np.ones(shape = (image_dimension,image_dimension,3)).astype('uint8')
    mask = 255*np.ones(shape = (image_dimension,image_dimension,3)).astype('uint8')
    for x,y,target,path in path_df[['x_coord','y_coord','target','path']].values:
        img_array = io.imread(path)
        # some patches have dimension less than 50 x 50 which would cause the code to break
        try:
            # replace values in grid array by the image array values
            grid[y:y+50,x:x+50] = img_array 
            # if the image is cancerous add
            if target != 0:
                mask[y:y+50,x:x+50] = [0,0,255]
        except: pass
    # check if prediction is specified
    if pred == False:
        img = grid
    else:
        alpha = 0.8
        # This is step is very important, multiplying the 2 values by a float value converts the arrays to float64, which is why convert them back to uint8
        img  = (mask * (1.0 - alpha) + grid * alpha).astype('uint8')
        
    return img

In [85]:
def plot_tissue_images(nrows=3,ncols=3,pred = False,df=df):
    n_imgs = nrows*ncols
    p_ids = np.random.choice(df['patient_id'].unique(), size=n_imgs, replace=False)
    
    fig,axs = plt.subplots(nrows=nrows,ncols=ncols,figsize=(30,30))
    for i,row in enumerate(axs):
        for j,ax in enumerate(row):
            p_id = p_ids[i*nrows+j]
            img = get_tissue_image_array(p_id,pred = pred)
            ax.set_title(f'Breast Tissue Slice for patient {p_id}')
            ax.set_xticks([])
            ax.set_yticks([])
            ax.imshow(img)

NameError: name 'df' is not defined

In [86]:
plot_tissue_images(pred = True)


NameError: name 'plot_tissue_images' is not defined

## taking sample of our data

In [87]:
# function that accepts number of patients and returns a dataframe containing all the images for each patient. 
# prioritizes patients with higher number of patches.
def get_patients(df,n_patients=10):
    p_ids = df.groupby('patient_id')['patient_id'].count().sort_values(ascending=False).index[0:n_patients]
    df =  df.loc[df['patient_id'].isin(p_ids)].reset_index(drop=True)
    return df

In [88]:
sample_df= get_patients(df=df)
sample_df.info()

NameError: name 'df' is not defined

In [89]:
def get_img_arrays(df,):
    # read each image array from corresponding path as grayscale and flatten the image array
    df['img_array'] = df.progress_apply(lambda x : io.imread(x['path'],as_gray=True).flatten(),axis=1) # make sure to specify axis = 1
    # get the shape of each image array and store it in the dataframe
    df['array_shape'] = df.progress_apply(lambda x : x['img_array'].shape[0],axis=1) # make sure to specify axis = 1
    return df

In [90]:
sample_df = get_img_arrays(df = sample_df)

NameError: name 'sample_df' is not defined

In [91]:
# get the count of the unique values in the column array_shape
sample_df['array_shape'].value_counts() 

NameError: name 'sample_df' is not defined

In [92]:
# get all images that do not have an array shape of 2500
weird_imgs = sample_df[sample_df['array_shape'] != 2500] 

NameError: name 'sample_df' is not defined

In [93]:
# use the show patches function that we created earlier to visualize those images
show_patches(nrows = 2,ncols = 5, path_df = weird_imgs)

NameError: name 'show_patches' is not defined

### Applying PCA on to the Images

In [94]:
# get the number of rows in the pandas dataframe in order to determine the number of rows in our numpy array
nrows=sample_df.shape[0]
# set the number of columns to 2500, which is the length of our array
ncols=2500
# initialize the array using the information above
img_arrays= np.zeros((nrows,ncols))
print(img_arrays.shape)

NameError: name 'sample_df' is not defined

In [95]:
# add all the image arrays to the numpy array that we just initialized
for i,array in enumerate(sample_df['img_array']):
    img_arrays[i,:] = array
display(img_arrays[0:5,:])

NameError: name 'sample_df' is not defined

In [96]:
from sklearn.decomposition import PCA
images_pca = PCA()
# fit the function to our image arrays
images_pca.fit(img_arrays);

Exception ignored in: <function tqdm.__del__ at 0x00000209C3A9E8C8>
Traceback (most recent call last):
  File "c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py", line 1131, in __del__
    self.close()
  File "c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\tqdm\notebook.py", line 261, in close
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


NameError: name 'img_arrays' is not defined

In [97]:
# explained get the cumalitive sum of the explained variance ratio for each principle component
evr = np.cumsum(images_pca.explained_variance_ratio_)
# plot the explained variance ratio
fig = go.Figure(data=go.Scatter(y = evr,line=dict(color='#ff70db')))
fig.update_layout(title='Explained Variance Ratio After PCA',
                   xaxis_title='Number of Principle Components',
                   yaxis_title='Cumalitive Explained Variance Ratio')
fig.show()

AttributeError: 'PCA' object has no attribute 'explained_variance_ratio_'

In [98]:
# let's now apply PCA but only retrieve the first 150 components or the components that account for 80 percent of the variance
images_pca = PCA(0.8)
# OR
images_pca = PCA(150)
# fit the function to our image arrays
images_pca.fit(img_arrays);

NameError: name 'img_arrays' is not defined

In [99]:
# get the shape of the first image array
sample_df['img_array'][0].shape

NameError: name 'sample_df' is not defined

In [100]:
# Apply PCA transformation to each row in the img_array column
# hint: the transform function accepts a list, so if you wanted to feed it one value you would have to place that value in a list
sample_df['pca_array'] = sample_df['img_array'].progress_apply(lambda x: images_pca.transform([x]).flatten()) 

NameError: name 'sample_df' is not defined

In [102]:
sample_df['pca_array'][0].shape
sample_df.head()


NameError: name 'sample_df' is not defined

In [103]:
# visualizing the first 30 principle components
fig, axes = plt.subplots(3, 10, figsize=(30, 10))
for i, ax in enumerate(axes.flat):
    # return each of the components, and reshape them to 50x50
    ax.imshow(images_pca.components_[i].reshape(50, 50), cmap='binary_r')

NameError: name 'plt' is not defined

In [104]:
components = images_pca.transform(img_arrays)
projected = images_pca.inverse_transform(components)

# Plot the results
fig, ax = plt.subplots(2, 10, figsize=(30, 7))
for i in range(10):
    ax[0, i].imshow(img_arrays[i].reshape(50, 50), cmap='binary_r')
    ax[1, i].imshow(projected[i].reshape(50, 50), cmap='binary_r')
    
ax[0, 0].set_ylabel('Before PCA')
ax[1, 0].set_ylabel('150-dim\nReconstruction');

NameError: name 'img_arrays' is not defined

In [105]:
# get the count of each of the target values
sample_df['target'].value_counts() 

NameError: name 'sample_df' is not defined

## train and test split

#### undersampling

In [106]:
# get the dataframe containing the negative target variables
negative = sample_df[sample_df['target'] == 0]
# get the dataframe containing the positive target variables
positive = sample_df[sample_df['target'] == 1]
# get the shapes of each dataframe
display(negative.shape)
display(positive.shape)

NameError: name 'sample_df' is not defined

In [107]:
from sklearn.utils import resample
# downsample the negative targets
neg_downsampled = resample(negative,n_samples=positive.shape[0], random_state=42)
# combine minority and downsampled majority
downsampled = pd.concat([positive, neg_downsampled])
# check new class counts
downsampled['target'].value_counts()

NameError: name 'negative' is not defined

In [108]:
# show the first 5 values of the dataframe
downsampled.head()

NameError: name 'downsampled' is not defined

In [109]:
# let's extract our variables of interest and store them in a new dataframe
dfd = downsampled.loc[:,['img_array','pca_array','target']]

NameError: name 'downsampled' is not defined

In [110]:
# get the number of rows in the pandas dataframe in order to determine the number of rows in our numpy array
nrows=dfd.shape[0]
# set the number of columns to 150, which is the length of our array
ncols=150
# initialize the array using the information above
pca_arrays= np.zeros((nrows,ncols))
print(pca_arrays.shape)

NameError: name 'dfd' is not defined

In [111]:
# loop over the array and replace the data
for i,array in enumerate(dfd['pca_array']):
    pca_arrays[i,:] = array

NameError: name 'dfd' is not defined

In [112]:
from sklearn.model_selection import train_test_split
# split our data into training and testing data, and input data and target data
X_train, X_test, y_train, y_test =  train_test_split(pca_arrays, dfd['target'], train_size=0.7, shuffle = True)

NameError: name 'pca_arrays' is not defined

In [113]:
# compare the shape of the train and test inputs
print(f'X_train Shape: {X_train.shape}')
print(f'X_test Shape: {X_test.shape}')
print(f'y_train Shape: {y_train.shape}')
print(f'y_test Shape: {y_test.shape}')

NameError: name 'X_train' is not defined

## training

In [114]:
from sklearn.svm import SVC
svc_rbf = SVC(kernel = 'rbf',gamma = 'auto' )
svc_linear = SVC(kernel='linear',gamma = 'auto')

In [115]:
# fitting our models
svc_rbf.fit(X = X_train,y = y_train);
svc_linear.fit(X = X_train,y = y_train);

NameError: name 'X_train' is not defined

In [116]:
print(svc_linear.score(X_train,y_train))
print(svc_rbf.score(X_train,y_train))

NameError: name 'X_train' is not defined

In [117]:
print(svc_linear.score(X_test,y_test))
print(svc_rbf.score(X_test,y_test))

NameError: name 'X_test' is not defined

## Confusion matrix

In [118]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_true=y_test,y_pred=svc_linear.predict(X_test)).ravel()

print(f'training set: true negatives: {tn}')
print(f'training set: true positives: {tp}')
print(f'training set: false negatives: {fn}')
print(f'training set: false positives: {fp}')

NameError: name 'y_test' is not defined

In [120]:
def confusion_vectors(predicted,actual):
    '''
    Function returns:
        # 0 False Negative
        # 1 True  Negative
        # 3 True  Positive
        # 4 False Positive
    '''
    # get the predicted values
    evaluation = (predicted + 1)**2 - actual
    return evaluation

In [121]:
# predict the values for each row, make sure to reshape since predict accepts a 2d vector 
# use [0] to return the first value since predict returns an array, and we are only predicting one row at a time
sample_df['predicted']=sample_df['pca_array'].progress_apply(lambda x: svc_rbf.predict(x.reshape(1,150))[0])

NameError: name 'sample_df' is not defined

In [122]:
# get the confusion vector values for each row, so that we can distinguish what type of error we got for each patch prediction
sample_df['conf']=sample_df.progress_apply(lambda x: confusion_vectors(predicted=x['predicted'],actual=x['target']),axis=1)

NameError: name 'sample_df' is not defined

In [123]:
def get_tissue_image_array_conf(patient_id,path_df):
    # get patient dataframe
    p_df = get_patient_df(patient_id,df=path_df)
    # get the max_coordinate to define the numpy array size, numpy array will be containing the rgb values for the image
    max_coord = np.max((*p_df['x_coord'],*p_df['y_coord']))
    # add 50 to the max_coord to get the final image dimension,50 here is the patch size
    image_dimension = max_coord + 50
    # create a 3 dimensional array with RGB values = 255, we set the type as uint since no RGB value can be > 255, thus increasing efficiency
    grid = 255*np.ones(shape = (image_dimension,image_dimension,3)).astype('uint8')
    # initialize empty array for for the mask
    mask = 255*np.ones(shape = (image_dimension,image_dimension,3)).astype('uint8')
    # create a dict containing the values to be filled in for each corresponding case
    # 0 is false negative, 3 is true positive, 4 is false positive
    mask_color = {0:[255,0,0],3:[0,0,255],4:[0,255,0]}

    for x,y,conf,path in p_df[['x_coord','y_coord','conf','path']].values:
        img_array = io.imread(path)
        # some patches have dimension less than 50 x 50 which would cause the code to break
        try:
            # replace values in grid array by the image array values
            grid[y:y+50,x:x+50] = img_array 
            # check if conf is not a true negative
            if conf != 1:
                mask[y:y+50,x:x+50] = mask_color[conf]
        except: pass

    alpha = 0.8
    # This is step is very important, multiplying the 2 values by a float value converts the arrays to float64, which is why convert them back to uint8
    img  = (mask * (1.0 - alpha) + grid * alpha).astype('uint8')
    
    return img

In [124]:
def plot_tissue_images_conf(nrows=3,ncols=3,df=df):
    n_imgs = nrows*ncols
    p_ids = np.random.choice(df['patient_id'].unique(), size=n_imgs, replace=False)
    
    fig,axs = plt.subplots(nrows=nrows,ncols=ncols,figsize=(30,30))
    for i,row in enumerate(axs):
        for j,ax in enumerate(row):
            p_id = p_ids[i*nrows+j]
            img = get_tissue_image_array_conf(p_id,path_df=df)
            ax.set_title(f'Breast Tissue Slice for patient {p_id}')
            ax.set_xticks([])
            ax.set_yticks([])
            ax.imshow(img)

NameError: name 'df' is not defined

In [125]:
plot_tissue_images_conf(df=sample_df)


NameError: name 'plot_tissue_images_conf' is not defined